## Introduction

In this code we plan on modeling stocks as a Markov chain. The first step in this pursuit will be importing stock data. I used the code below when constructing this script.

```python
import numpy as np
import pandas as pd
from urllib.request import urlopen
import re

alphaVantage_api_key = "user_key"
stock_Symbol = "HPE"
HPEurl = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=" + stock_Symbol + "&datatype=" + "csv" + "&outputsize="+ "full" +"&apikey=" + user_key
HPEcsv = urlopen(HPEurl)
HPEcurrentDF = pd.read_csv(HPEcsv, index_col = 0)
HPEcurrentDF = HPEcurrentDF.iloc[::-1] ```

This code uses a free api from [ALPHA VANTAGE](https://www.alphavantage.co). To use this api you must also get a user key. I have left out my own key and thus running this code will result in an error. To circumvent this I shall import data directly from a file and provide that file. (Note: the csv file received from alpha vantage is written from earliest to latest. The code "FMCCcurrentDF = FMCCcurrentDF.iloc[::-1]" reverses the order). 

Before we begin with the code I wish to state which variables the user should feel free to change. These variables can easily be used as constructors in a class if needed. The variables are
```python
HPEcsv = #csv file of your choosing#
col = #Must match a numerical attribute of the csv file#
index_col = #Can be changed to match the index of your csv file#
time_Interval = #Can be changed to average the number of days you wish#
given_State = #Can be changed to your state of interest#
```

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jul  8 19:03:46 2019

@author: pbraz
"""
import numpy as np
import pandas as pd
from urllib.request import urlopen
import re

HPEcsv = "Directory_Of_File"
HPEdf = pd.read_csv(HPEcsv, index_col = 0)
print(HPEdf.head())

                 Open       High       Low     Close  Adj Close   Volume
Date                                                                    
2015-10-19  10.752331  10.752331  9.578293  9.618977   8.884633  2721200
2015-10-20   9.444614   9.839835  9.444614  9.804963   9.056418  1220800
2015-10-21  10.025822  10.083942  9.531796  9.578293   8.847054   135200
2015-10-22   9.711970   9.909580  9.438803  9.851460   9.099369   189200
2015-10-23  10.025822  10.025822  9.793339  9.880520   9.126210   102900


Here I imported a file from my directory as a dataframe using pandas (the file was downloaded from [yahoo finance](https://finance.yahoo.com/quote/HPE/history?period1=1445227200&period2=1564632000&interval=1d&filter=history&frequency=1d)). I then printed the first five rows using the dot head method to get a sense of the file. Only one of these columns are necessary for our purposes.

In [3]:
col = "Open"
stockRef = HPEdf[col]
df_Size = len(stockRef)

We extract the "Open" column from the dataframe, assign the series to a variable and record it's length.

In [4]:
percent_change= stockRef.copy()

for i in range(0, df_Size-1):
    percent_change.iloc[i+1] = (stockRef.iloc[i+1] / stockRef.iloc[i])*100 - 100

percent_change.iloc[0] = 0.

Since we modeling this problem as a Markov chain we need to define our states. Using each opening value as a state would be fruitless. At 7 decimal places of precision each record would be it's own state. What's more there are plenty of values that will, more than likely, never be reached again (such as the minimum recorded value). Our states shall be the percentage change in the value of the stock from the previous time interval. In the code above we create a copy of our series without worrying about changing the original. Our new series records how much the percentage change in the stock's value from the previous day.

In [5]:
time_Interval = 1 #Measured in Days#

percent_Avg = []

percent_time_Size =int(df_Size / time_Interval)

for i in range(0, percent_time_Size):
    current_Avg = percent_change.iloc[(i)*(time_Interval) : (i+1)*(time_Interval)].mean()
    percent_Avg.append(current_Avg)

Here we give the option to condense days by averaging them. One might be more interested in the average weekly change in a stock's value rather than daily. Here we average the percentage change over the past "time_Interval" days to give a sense of how the value of the stock changed over that time.

In [6]:
percent_Avg_Binned = map(lambda x: int(x), percent_Avg)
percent_Avg_Binned = list(percent_Avg_Binned)#Measured in Percent#

We still face the problem of our state space being too granular. We wish for each state to appear a reasonable number of times (certainly more than once). We solve this problem by putting the percentages into bins. For example, if the percentage change over the past week averaged at 7.42456 we can round that off to 7. Here I change all values into integers.

In [9]:
unique_States = set(percent_Avg_Binned)
unique_States = list(unique_States)
state_Size = len(unique_States)

By converting our data into a set we find all the unique values. This is our state space for the Markov chain. Finally we determine the number of states and assign it to "state_Size".

In [10]:
state_Dic = {}
for i in range(0, state_Size):
    state_Dic[i] = str(unique_States[i])
    
print(state_Dic)

{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '13', 11: '-12', 12: '-10', 13: '-2', 14: '-8', 15: '-7', 16: '-6', 17: '-5', 18: '-4', 19: '-3', 20: '-1'}


We create a dictionary to associate each state with it's future integer position. We then print the dictionary so the user has a sense of the state space.

In [11]:
string_Search = str(percent_Avg_Binned)
    
markov_Transition_Rel_F = np.zeros((state_Size, state_Size))

for initial_State, initial_State_Name in state_Dic.items():
    for end_State, end_State_Name in state_Dic.items():
        pattern = initial_State_Name + ", " + end_State_Name
        markov_Transition_Rel_F[initial_State][end_State] = len(re.findall(pattern, string_Search))

for state, state_Name in state_Dic.items():
    pattern = state_Name
    freq_State = len(re.findall(pattern, string_Search))
    markov_Transition_Rel_F[state] = markov_Transition_Rel_F[state] / freq_State

Now that our states are defined we need to figure out the transition probabilities. To do this we recast our data as a string and find the number of times we observe 'initial_State, end_State". This will tell us the frequence of transitions from "initial_State" to "end_State". After this is recorded in our "state_Size" by "state_Size" matrix (2D-array) we divide each transition from "initial_State" to "end_State" by the frequency of "initial_State". This gives us the relative frequency from "initial_State" to "end_State" and completes the transition matrix. 

In [12]:
given_State = 15

try:
    given_State in unique_States
except:
    print("Value is not in the state space!")

for state, state_Name in state_Dic.items():
    if str(given_State) ==  state_Name:
        given_State = state

most_Likely_State = np.argmax(markov_Transition_Rel_F[given_State])
belief = markov_Transition_Rel_F[given_State][most_Likely_State]
print("Given you are in state " + str(given_State) + " you are most likely to transition to state " + str(most_Likely_State) + " with a measure of belief of " + str(belief))

Given you are in state 15 you are most likely to transition to state 2 with a measure of belief of 0.666666666667
